# Предисловие

- Задание ДЗ3: выгрузить данные и сформировать из них dataframe в pandas
- Задание ДЗ3* (доп): кто хочет продвигаться в анализе данных, советую сделать анализ выгруженных данных с помощью pandas

![drakkar image](https://cs8.pikabu.ru/post_img/big/2016/06/15/6/146598388513055440.png)

это третье домашнее задание (второе дз было на повторение линукса - самостоятельно, я не буду его проверять)


## Задание №1

Скачайте файлы **Version 1** Steam data from [Recommender Systems Datasets](http://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data). 

нам нужны файлы:
- Version 1: Review Data 
- Version 1: User and Item Data 

Ваша задача развернуть данные в pd.DataFrame (ДЗ3)

In [2]:
import pandas as pd
import numpy as np
import ast
from pandas.io.json import json_normalize

In [16]:
data = []
with open("australian_user_reviews.json",'r', encoding ='utf-8') as file:
    for line in file.readlines():
        data.append(ast.literal_eval(line))

Теперь, сделаем из распакованных данных, датафрейм.

In [17]:
reviews = json_normalize(data)

Посмотрим какой размер у датафрейма

In [18]:
reviews.shape

(25799, 3)

Теперь посмотрим на заголовки и на то как, примерно выглядит наш датафрейм. И мы видим, что в колонке reviews, стоят данные вперемешку для одного игрока.

In [5]:
reviews.head()

,reviews,user_id,user_url
0,"[{'funny': '', 'posted': 'Posted November 5, 2...",76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"[{'funny': '', 'posted': 'Posted June 24, 2014...",js41637,http://steamcommunity.com/id/js41637
2,"[{'funny': '', 'posted': 'Posted February 3.',...",evcentric,http://steamcommunity.com/id/evcentric
3,"[{'funny': '', 'posted': 'Posted October 14, 2...",doctr,http://steamcommunity.com/id/doctr
4,"[{'funny': '3 people found this review funny',...",maplemage,http://steamcommunity.com/id/maplemage


Для reviews стоит немного предобработать данные, код ниже (следующие 2 блока кода возможно вам не понадобится, всё будет зависеть от того, как вы развернете данные - смотрите сами крч). 

Тут получаем датафрейм отдельно для обзоров.

In [6]:
# drop duplicated user IDs
reviews = reviews.drop_duplicates(subset='user_id')
# drop users with no reviews
reviews = reviews[reviews['reviews'].apply(len) > 0]

Для начала посмотрим, поменялись ли размеры, после того как мы убрали повторяющиеся id, и id без обзоров.

In [7]:
reviews.shape

(25457, 3)

получим уже отдельный датафрейм для обзоров

In [8]:
def unpack_user_reviews(row):
    "Unpack items from the items in a user's row"
    # items is a series of lists - chain will make one long iterable
    # convert this to a data frame
    rdf = pd.DataFrame.from_records(row.reviews)
    rdf['user_id'] = row.user_id
    return rdf

game_reviews = pd.concat((unpack_user_reviews(row) for row in reviews.itertuples()),
                         ignore_index=True)

In [10]:
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [9]:
game_reviews.shape

(58430, 8)

Теперь проделаем аналогичные операции для второго файла, содержащего данные, о том, кто в какие игры играл, и т.п.

In [3]:
data1 = []
with open("australian_users_items.json",'r', encoding ='utf-8') as file:
    for line in file.readlines():
        data1.append(ast.literal_eval(line))

In [4]:
items = json_normalize(data1)

In [5]:
items.head()

,items,items_count,steam_id,user_id,user_url
0,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
2,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137,76561198007712555,evcentric,http://steamcommunity.com/id/evcentric
3,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328,76561197963445855,Riot-Punch,http://steamcommunity.com/id/Riot-Punch
4,"[{'item_id': '300', 'item_name': 'Day of Defea...",541,76561198002099482,doctr,http://steamcommunity.com/id/doctr


In [6]:
items.shape

(88310, 5)

In [7]:
# drop duplicated user IDs
items = items.drop_duplicates(subset='user_id')
# drop users with no items
items = items[items['items'].apply(len) > 0]

In [8]:
items.shape

(70912, 5)

In [9]:
def unpack_user_items(row):
    
    rdf1 = pd.DataFrame.from_records(row.items)
    rdf1['user_id'] = row.user_id
    return rdf1

game_items = pd.concat((unpack_user_items(row) for row in items.itertuples()),
                         ignore_index=True)

In [10]:
game_items.head()

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0,6,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,0,7,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


## Подсчитайте следующие статистики: ДЗ3*
### Q1: Количество уникальных юзеров:

In [10]:
reviews.shape


(25457, 3)

Так как мы удаляли повторяющиеся id, то количество уникальных это то сколько строк в reviews, т.е 25457

### Q2: Количество уникального контента:

In [11]:
cont = game_reviews['item_id']

In [12]:
cont.unique()
print(len(cont.unique()))

3682


Таким образом видим,что уникального контента 3682

### Q3: Найдите самый смешной review, рекомендовали ли там игру?


In [40]:
f = game_reviews['funny']
f = list(f)
for i in range(58430):
    if f[i] != '':
        f[i] = str(f[i])
        f[i] = f[i].split()
        f[i] = f[i][:1]
        f[i] = int(f[i][0].replace(',',''))
    else: 
        f[i] = 1


M = max(f)
index=[]
for i in range(len(f)):
    if f[i] == M:
        index.append(i)

print(index[0], '/индекс самого смешного')
print(M, "/максимально количество людей, посчитавших, этот обзор смешным")
print(f[index[0]],"/проверили, что элемент с этим индексом, действительно максимум")
print(game_reviews['recommend'].iloc[index[0]],'/была ли рекомендована игра?')

21895 /индекс самого смешного
2598 /максимально количество людей, посчитавших, этот обзор смешным
2598 /проверили, что элемент с этим индексом, действительно максимум
True /была ли рекомендована игра?


Таким образом, видим, что самый смешной обзор, это элемент 21895, и игра была рекомендована.

### Q4: Уникальные пользователи:
- Сколько пользователей купили все игры;
- Сколько пользователей рекомендовали(recommend:True) все игры.

In [ ]:
# Your code should be written below


### Q5: Посмотрим на активность пользователей: 
#### Постройте следующие гистограммы: 
- Кол-во приобретенных игр на юзера;
- Кол-во оставленных "review" на юзера.

In [ ]:
# Your code should be written below


## Важно!
#### Всё задание можно выполнить за 45 минут  (но это не точно) с учетом прогрузки данных! Если Ваш код работает дольше, то Вы что-то делаете не правильно.  
### Удачи.